In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint

In [ ]:
# Load metadata
metadata_path = 'data/metadata.csv'
metadata = pd.read_csv(metadata_path)

# Load and preprocess images
def load_and_preprocess_images(image_paths, target_size=(256, 256)):
    images = []
    for path in image_paths:
        img = tf.image.decode_jpeg(tf.io.read_file(path))
        img = tf.image.resize(img, target_size)
        img = img / 255.0
        images.append(img)
    return np.array(images)

image_paths = ['data/' + img_id + '.jpg' for img_id in metadata['isic_id']]
images = load_and_preprocess_images(image_paths)

# Prepare target masks
mask_paths = ['data/masks/' + img_id + '_mask.jpg' for img_id in metadata['isic_id']]
masks = load_and_preprocess_images(mask_paths)

# Split the data into training and testing sets
train_images, test_images, train_masks, test_masks = train_test_split(images, masks, test_size=0.2, random_state=42)

In [ ]:
def unet_model(input_shape, num_channels=3):
    inputs = Input(input_shape)

    # Contracting Path (Encoder)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)

    # Expanding Path (Decoder)
    up6 = UpSampling2D(size=(2, 2))(drop5)
    up6 = Conv2D(512, 2, activation='relu', padding='same')(up6)
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = UpSampling2D(size=(2, 2))(conv6)
    up7 = Conv2D(256, 2, activation='relu', padding='same')(up7)
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = UpSampling2D(size=(2, 2))(conv7)
    up8 = Conv2D(128, 2, activation='relu', padding='same')(up8)
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = UpSampling2D(size=(2, 2))(conv8)
    up9 = Conv2D(64, 2, activation='relu', padding='same')(up9)
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

    # Output Layer
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    # Build the model
    model = Model(inputs, outputs)
    return model

# Create the U-Net model
input_shape = (256, 256, 3)  # RGB images
model = unet_model(input_shape)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Data augmentation with moderate techniques
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect'
)

# Early Stopping
early_stopping = EarlyStopping(patience=25, restore_best_weights=True)

# Learning rate schedule
def lr_schedule(epoch):
    if epoch < 20:
        return 0.001
    elif epoch < 40:
        return 0.0001
    else:
        return 0.00001

lr_scheduler = LearningRateScheduler(lr_schedule)

In [ ]:
# Compile the model (already compiled, you can remove this part)
model = unet_model(input_shape=(256, 256, 3))
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Load pre-trained model weights
model.load_weights('model_weights.h5') 

# Retrain with the complete dataset and fine-tuned hyperparameters
batch_size = 16  # Experiment with batch size
epochs = 50  # Adjust as needed

history = model.fit(
    datagen.flow(train_images, train_masks, batch_size=batch_size),
    steps_per_epoch=len(train_images) // batch_size,
    epochs=epochs,
    validation_data=(test_images, test_masks),  # Use a validation split to monitor performance
    callbacks=[early_stopping, lr_scheduler])

In [ ]:
# Clear previous model and release memory
tf.keras.backend.clear_session()

# Update learning rate schedule
def lr_schedule(epoch):
    initial_lr = 0.001
    decay_rate = 0.5
    decay_steps = 10
    new_lr = initial_lr * (decay_rate ** (epoch // decay_steps))
    return new_lr

lr_scheduler = LearningRateScheduler(lr_schedule)

# Update batch size and epochs
batch_size = 18  # Experiment with different batch sizes
epochs = 100    # Increase the number of epochs

# Increase early stopping patience
early_stopping = EarlyStopping(patience=50, restore_best_weights=True)

In [ ]:
# Create a ModelCheckpoint callback
model_checkpoint = ModelCheckpoint(
    'updated_model_weights.h5',  # Path to save the updated weights
    monitor='val_loss',          # Monitor validation loss
    save_best_only=True,         # Save only the best model
    mode='min'                   # Mode can be 'min' or 'max' depending on the monitored metric
)

# Load pre-trained model weights
model.load_weights('model_weights.h5') 

# Start retraining
history = model.fit(
    datagen.flow(train_images, train_masks, batch_size=batch_size),
    steps_per_epoch=len(train_images) // batch_size,
    epochs=epochs,
    validation_data=(test_images, test_masks),  
    callbacks=[early_stopping, lr_scheduler, model_checkpoint])

In [ ]:
import sklearn.metrics as metrics

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_images, test_masks, batch_size=batch_size)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

# Predict masks on the test set
test_predictions = model.predict(test_images)

# Convert the predictions and labels to NumPy arrays
predictions = np.array(test_predictions)
labels = np.array(test_masks)

# Plot the predictions against the actual labels
plt.scatter(predictions, labels)
plt.xlabel("Predictions")
plt.ylabel("Actual Labels")
plt.title("Predictions vs. Actual Labels")
plt.show()